In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
from tensorflow import keras
from functools import partial

In [3]:
root_logdir = os.path.join(os.curdir, "my_tb_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [4]:
with tf.device("/cpu:0"):
    dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
    dataset_size = info.splits["train"].num_examples
    class_names = info.features["label"].names
    n_classes = info.features["label"].num_classes
    
    test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)
    def central_crop(image):
        shape = tf.shape(image)
        min_dim = tf.reduce_min([shape[0], shape[1]])
        top_crop = (shape[0] - min_dim) // 4
        bottom_crop = shape[0] - top_crop
        left_crop = (shape[1] - min_dim) // 4
        right_crop = shape[1] - left_crop
        return image[top_crop:bottom_crop, left_crop:right_crop]

    def random_crop(image):
        shape = tf.shape(image)
        min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
        return tf.image.random_crop(image, [min_dim, min_dim, 3])
    def preprocess(image, label, randomize=False):
        if randomize:
            cropped_image = random_crop(image)
            cropped_image = tf.image.random_flip_left_right(cropped_image)
        else:
            cropped_image = central_crop(image)
        resized_image = tf.image.resize(cropped_image, [224, 224])
        final_image = keras.applications.xception.preprocess_input(resized_image)
        return final_image, label

    batch_size = 32
    train_set = train_set_raw.shuffle(1000).repeat()
    train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
    valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
    test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)
    base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
    avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
    output = keras.layers.Dense(n_classes, activation="softmax")(avg)
    model = keras.models.Model(inputs=base_model.input, outputs=output)
    for layer in base_model.layers:
        layer.trainable = False
    optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [5]:
with tf.device("/cpu:0"):
    history = model.fit(train_set, steps_per_epoch=int(0.75 * dataset_size / batch_size), validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size), epochs=5)

Epoch 1/5
86/86 [==============================] - 216s 2s/step - loss: 1.2762 - accuracy: 0.8016 - val_loss: 1.0863 - val_accuracy: 0.8364
Epoch 2/5
86/86 [==============================] - 217s 3s/step - loss: 0.5435 - accuracy: 0.8964 - val_loss: 0.7700 - val_accuracy: 0.8658
Epoch 3/5
86/86 [==============================] - 216s 3s/step - loss: 0.3301 - accuracy: 0.9208 - val_loss: 0.7801 - val_accuracy: 0.8548
Epoch 4/5
86/86 [==============================] - 215s 3s/step - loss: 0.2236 - accuracy: 0.9375 - val_loss: 0.6437 - val_accuracy: 0.8713
Epoch 5/5
86/86 [==============================] - 216s 3s/step - loss: 0.1928 - accuracy: 0.9455 - val_loss: 0.6261 - val_accuracy: 0.8750


In [ ]:
tf.config.experimental.get_memory_info('GPU:0')

In [6]:
for layer in base_model.layers:
    layer.trainable=True
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=int(0.75 * dataset_size / batch_size), validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size), epochs=5, callbacks=[tensorboard_cb])

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'model/block3_sepconv2_bn/FusedBatchNormV3' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
    

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
history = model.fit(train_set, steps_per_epoch=int(0.75 * dataset_size / batch_size), validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size), epochs=35, callbacks=[tensorboard_cb, early_stopping_cb])

In [ ]:
model.evaluate(test_set)

In [ ]:
tf.config.experimental.get_memory_info('GPU:0')